# ETL Pipeline: PDF to PostgreSQL pgvector

이 노트북은 PDF 파일을 읽어서 마크다운으로 변환하고, 청크로 나눈 후 임베딩하여 PostgreSQL pgvector에 적재하는 전체 ETL 파이프라인을 구현합니다.

## Pipeline Overview
1. **Chapter-based Markdown Conversion**: 챕터 배열을 받아서 챕터별로 나눈 후 마크다운으로 변환
2. **Noise Removal**: 챕터 첫 페이지와 전체 페이지에서 노이즈 제거 (header/footer 패턴)
3. **Markdown Header Labeling**: 특정 패턴(Exercises, Key Terms 등)을 마크다운 헤더로 변환
4. **Header-based Chunking**: 마크다운 헤더를 기준으로 청크 분할
5. **Header-based Filtering**: 특정 헤더를 가진 청크 필터링
6. **Embedding with Clova**: Clova 임베딩 API를 사용한 벡터 생성 (RPM 고려)
7. **PostgreSQL pgvector Loading**: 임베딩 결과를 PostgreSQL pgvector에 적재

---
## Checkpoint 1: Chapter-based Markdown Conversion

PDF 파일에서 챕터 정보를 받아 각 챕터별로 페이지를 나누고 마크다운으로 변환합니다.

### 입력
- PDF 파일 경로
- 챕터별 시작 페이지 정보 (딕셔너리)

### 출력
- `chapter_markdowns`: 챕터별 마크다운 텍스트 (dict)

### 체크포인트 저장
- 변수: `checkpoint_1_chapter_markdowns`

### Step 1.1: 라이브러리 설치

필요한 라이브러리를 설치합니다.

In [ ]:
!pip install pymupdf4llm pymupdf -q

### Step 1.2: 라이브러리 임포트

필요한 라이브러리를 임포트합니다.

In [ ]:
import pymupdf4llm
import fitz  # PyMuPDF
import os

### Step 1.3: PDF 파일 및 챕터 정보 설정

처리할 PDF 파일 경로와 각 챕터의 시작 페이지 번호를 정의합니다.

In [ ]:
# PDF 파일 경로
pdf_path = "data/network.pdf"

# 챕터별 시작 페이지 번호 (PDF 페이지 번호)
chapter_start_pages = {
    "Chapter 1": 5,
    "Chapter 2": 47,
    "Chapter 3": 103,
    "Chapter 4": 179,
    "Chapter 5": 229,
    "Chapter 6": 287,
    "Chapter 7": 341,
    "Chapter 8": 373,
    "Chapter 9": 415,
}

# 챕터 정보 정렬 및 총 페이지 수 확인
sorted_chapters = sorted(chapter_start_pages.items(), key=lambda item: item[1])

doc = fitz.open(pdf_path)
total_pages = len(doc)
doc.close()

print(f"Target PDF: {pdf_path}")
print(f"Total Pages: {total_pages}")
print(f"Total Chapters: {len(sorted_chapters)}")
print(f"\nChapter Configuration:")
for chapter_name, start_page in sorted_chapters:
    print(f"  - {chapter_name}: Page {start_page}")

### Step 1.4: 챕터별 마크다운 변환 실행

각 챕터의 페이지 범위를 계산하고 pymupdf4llm을 사용하여 마크다운으로 변환합니다.

In [ ]:
chapter_markdowns = {}

print("=" * 60)
print("Starting Chapter-based Markdown Conversion")
print("=" * 60)

for i, (chapter_name, start_page) in enumerate(sorted_chapters):
    # 페이지 범위 계산 (0-indexed)
    start_idx = start_page - 1
    
    if i < len(sorted_chapters) - 1:
        # 다음 챕터 시작 전까지
        end_idx = sorted_chapters[i + 1][1] - 1
    else:
        # 마지막 챕터는 PDF 끝까지
        end_idx = total_pages
    
    # 페이지 범위 리스트 생성
    page_range = list(range(start_idx, end_idx))
    
    print(f"\n[{chapter_name}]")
    print(f"  Processing pages {start_idx + 1} to {end_idx} ({len(page_range)} pages)...")
    
    # pymupdf4llm을 사용하여 페이지별로 마크다운 변환
    chapter_md_text = pymupdf4llm.to_markdown(pdf_path, pages=page_range)
    
    # 챕터별 마크다운 저장
    chapter_markdowns[chapter_name] = chapter_md_text
    
    print(f"  ✓ Completed (Length: {len(chapter_md_text):,} characters)")

print("\n" + "=" * 60)
print(f"✓ All chapters converted successfully!")
print(f"  Total chapters: {len(chapter_markdowns)}")
print("=" * 60)

### Step 1.5: 체크포인트 저장 및 결과 확인

변환된 마크다운을 체크포인트로 저장하고 샘플 결과를 확인합니다.

In [ ]:
# 체크포인트 1 저장
checkpoint_1_chapter_markdowns = chapter_markdowns.copy()

print("✓ Checkpoint 1 saved successfully!")
print(f"  Variable: checkpoint_1_chapter_markdowns")
print(f"  Chapters: {list(checkpoint_1_chapter_markdowns.keys())}")
print(f"\n--- Sample: First 500 characters of Chapter 1 ---")
print(checkpoint_1_chapter_markdowns["Chapter 1"][:3000])
print("...")

---
## Checkpoint 2: Noise Removal

변환된 마크다운에서 불필요한 노이즈를 제거합니다.
- 전체 페이지: Header/Footer 영역의 페이지 번호, Copyright 문구 등 제거
- 챕터 첫 페이지: 추가로 중복되는 챕터 제목 등 제거

### 입력
- `checkpoint_1_chapter_markdowns`: 챕터별 원본 마크다운

### 출력
- `chapter_markdowns_cleaned`: 노이즈가 제거된 챕터별 마크다운 (dict)

### 체크포인트 저장
- 변수: `checkpoint_2_cleaned_markdowns`

### Step 2.1: 라이브러리 임포트

정규표현식 처리를 위한 라이브러리를 임포트합니다.

In [ ]:
import re

### Step 2.2: 노이즈 제거 패턴 설정

In [ ]:
# Header/Footer 확인 범위 설정 (각각 몇 줄씩 체크할지)
header_check_range = 5
footer_check_range = 5

# Header/Footer 무조건 삭제할 줄 수 (패턴 무관)
header_remove_lines = 0     # 상단 N줄을 무조건 삭제 (0 = 삭제 안함)
footer_remove_lines = 4     # 하단 N줄을 무조건 삭제 (0 = 삭제 안함)

# Header 패턴 (페이지 상단에서 제거할 패턴)
header_patterns = [
    r"Computer Networks: A Systems Approach, Release Version 6.1"
]

# Footer 패턴 (페이지 하단에서 제거할 패턴)
footer_patterns = []

# 챕터 첫 페이지 전용 패턴 (첫 페이지에서만 추가로 제거)
first_page_patterns = []

print("Noise Removal Configuration:")
print(f"  Header check range: {header_check_range} lines")
print(f"  Header unconditional removal: {header_remove_lines} lines")
print(f"  Footer check range: {footer_check_range} lines")
print(f"  Footer unconditional removal: {footer_remove_lines} lines")
print(f"  Header patterns: {len(header_patterns)} patterns")
print(f"  Footer patterns: {len(footer_patterns)} patterns")
print(f"  First page patterns: {len(first_page_patterns)} patterns")

### Step 2.3: 노이즈 제거 함수 정의

In [ ]:
def clean_page_markdown(md_text, is_first_page=False, show_removed=False, page_num=None):
    """
    페이지별 마크다운에서 노이즈를 제거합니다.
    
    Args:
        md_text (str): 원본 마크다운 텍스트
        is_first_page (bool): 챕터의 첫 페이지 여부
        show_removed (bool): 제거된 줄을 표시할지 여부
        page_num (int): 페이지 번호 (로깅용)
        
    Returns:
        str: 노이즈가 제거된 마크다운 텍스트
    """
    if not md_text:
        return ""
    
    lines = md_text.split('\n')
    lines_to_remove = set()
    removal_reasons = {}  # 제거 이유 저장
    
    # 1. 무조건 삭제할 줄 처리
    # Header: 상단 N줄 무조건 삭제
    if header_remove_lines > 0:
        for i in range(min(header_remove_lines, len(lines))):
            lines_to_remove.add(i)
            removal_reasons[i] = f"Header unconditional removal (top {header_remove_lines} lines)"
    
    # Footer: 하단 N줄 무조건 삭제
    if footer_remove_lines > 0:
        footer_start = max(0, len(lines) - footer_remove_lines)
        for i in range(footer_start, len(lines)):
            lines_to_remove.add(i)
            removal_reasons[i] = f"Footer unconditional removal (bottom {footer_remove_lines} lines)"
    
    # 2. Header 영역에서 패턴 매칭
    for i in range(min(header_check_range, len(lines))):
        if i in lines_to_remove:
            continue  # 이미 제거 대상이면 스킵
        
        line_stripped = lines[i].strip()
        
        # Header 패턴 체크
        for pattern in header_patterns:
            if re.search(pattern, line_stripped, re.IGNORECASE):
                lines_to_remove.add(i)
                removal_reasons[i] = f"Header pattern: {pattern}"
                break
    
    # 3. Footer 영역에서 패턴 매칭
    footer_start_idx = max(0, len(lines) - footer_check_range)
    for i in range(footer_start_idx, len(lines)):
        if i in lines_to_remove:
            continue  # 이미 제거 대상이면 스킵
        
        line_stripped = lines[i].strip()
        
        # Footer 패턴 체크
        for pattern in footer_patterns:
            if re.search(pattern, line_stripped, re.IGNORECASE):
                lines_to_remove.add(i)
                removal_reasons[i] = f"Footer pattern: {pattern}"
                break
    
    # 4. 챕터 첫 페이지인 경우 추가 패턴 체크
    if is_first_page:
        for i, line in enumerate(lines):
            if i in lines_to_remove:
                continue
            line_stripped = line.strip()
            for pattern in first_page_patterns:
                if re.search(pattern, line_stripped, re.IGNORECASE):
                    lines_to_remove.add(i)
                    removal_reasons[i] = f"First page pattern: {pattern}"
                    break
    
    # 제거된 줄 표시
    if show_removed and lines_to_remove:
        print(f"    [Page {page_num}] Removed {len(lines_to_remove)} line(s):")
        for i in sorted(lines_to_remove)[:10]:  # 처음 10개만 표시
            reason = removal_reasons.get(i, "Unknown")
            line_preview = lines[i][:60] + "..." if len(lines[i]) > 60 else lines[i]
            print(f"      Line {i}: '{line_preview}' ({reason})")
        if len(lines_to_remove) > 10:
            print(f"      ... and {len(lines_to_remove) - 10} more lines")
    
    # 제거 대상이 아닌 줄만 필터링
    cleaned_lines = [line for i, line in enumerate(lines) if i not in lines_to_remove]
    
    return '\n'.join(cleaned_lines)


print("✓ Noise removal function defined successfully!")
print("  - Header: Unconditional removal + pattern matching")
print("  - Footer: Unconditional removal + pattern matching")
print("  - First page: Pattern matching")

### Step 2.4: 챕터별 노이즈 제거 실행

Checkpoint 1에서 변환된 마크다운에 노이즈 제거 함수를 적용합니다.
각 챕터는 여러 페이지로 구성되어 있으므로, 페이지 단위로 분리하여 처리합니다.

In [ ]:
chapter_markdowns_cleaned = {}

print("=" * 60)
print("Starting Noise Removal")
print("=" * 60)

for i, (chapter_name, start_page) in enumerate(sorted_chapters):
    # 페이지 범위 계산 (0-indexed)
    start_idx = start_page - 1
    
    if i < len(sorted_chapters) - 1:
        end_idx = sorted_chapters[i + 1][1] - 1
    else:
        end_idx = total_pages
    
    print(f"\n[{chapter_name}]")
    print(f"  Processing pages {start_idx + 1} to {end_idx}...")
    
    # 페이지별로 변환하고 노이즈 제거 적용
    chapter_full_text = []
    
    for page_idx in range(start_idx, end_idx):
        # 첫 페이지 여부 확인
        is_first_page = (page_idx == start_idx)
        
        # 페이지를 마크다운으로 변환
        page_md = pymupdf4llm.to_markdown(pdf_path, pages=[page_idx])
        
        # 노이즈 제거 적용 (제거된 줄 표시)
        cleaned_md = clean_page_markdown(
            page_md, 
            is_first_page=is_first_page, 
            show_removed=True, 
            page_num=page_idx + 1
        )
        
        chapter_full_text.append(cleaned_md)
    
    # 챕터의 모든 페이지를 결합
    chapter_markdowns_cleaned[chapter_name] = "\n\n".join(chapter_full_text)
    
    original_length = len(checkpoint_1_chapter_markdowns.get(chapter_name, ""))
    cleaned_length = len(chapter_markdowns_cleaned[chapter_name])
    removed = original_length - cleaned_length
    
    print(f"  ✓ Completed")
    print(f"    Original: {original_length:,} chars")
    print(f"    Cleaned: {cleaned_length:,} chars")
    print(f"    Removed: {removed:,} chars")

print("\n" + "=" * 60)
print(f"✓ Noise removal completed for all chapters!")
print(f"  Total chapters: {len(chapter_markdowns_cleaned)}")
print("=" * 60)

### Step 2.5: 체크포인트 저장 및 결과 확인

노이즈가 제거된 마크다운을 체크포인트로 저장하고 샘플 결과를 확인합니다.

In [ ]:
# 체크포인트 2 저장
checkpoint_2_cleaned_markdowns = chapter_markdowns_cleaned.copy()

print("✓ Checkpoint 2 saved successfully!")
print(f"  Variable: checkpoint_2_cleaned_markdowns")
print(f"  Chapters: {list(checkpoint_2_cleaned_markdowns.keys())}")
print(f"\n--- Sample: First 500 characters of cleaned Chapter 1 ---")
print(checkpoint_2_cleaned_markdowns["Chapter 9"][:1000])
print("...")

---
## Checkpoint 3: Markdown Header Labeling

특정 키워드(Exercises, Attribution, Key Terms 등)를 찾아서 마크다운 헤더로 변환합니다.

### 입력
- `checkpoint_2_cleaned_markdowns`: 정제된 챕터별 마크다운
- `header_labels`: 변환할 키워드와 헤더 레벨 매핑 (dict)

### 출력
- `chapter_markdowns_labeled`: 헤더가 라벨링된 챕터별 마크다운 (dict)

### 체크포인트 저장
- 변수: `checkpoint_3_labeled_markdowns`

### Step 3.1: 헤더 라벨링 설정

특정 키워드를 마크다운 헤더로 변환하기 위한 설정을 정의합니다.

In [ ]:
# 헤더로 변환할 키워드와 헤더 레벨 매핑
# 키: 찾을 키워드, 값: 헤더 레벨 (2 = ##)
header_labels = {
    "Key Takeaway": 1
}

print("Header Labeling Configuration:")
print(f"  Total keywords: {len(header_labels)}")
print(f"\nKeywords to convert:")
for keyword, level in header_labels.items():
    header_symbol = "#" * level
    print(f"  - '{keyword}' → '{header_symbol} {keyword}'")

### Step 3.2: 헤더 라벨링 함수 정의

키워드를 찾아서 마크다운 헤더로 변환하는 함수를 정의합니다.

In [ ]:
import re

def label_headers(text, label_map, show_conversions=False):
    if not text:
        return ""
    
    lines = text.split('\n')
    new_lines = []
    conversions = []
    
    for line_num, line in enumerate(lines):
        line_stripped = line.strip()
        matched = False
        
        # 1. 마크다운 특수 기호들 제거 (앞뒤에 붙는 것들 위주)
        cleaned_line = line_stripped.strip("#* _~`>-")
        
        # 2. HTML 태그가 섞여 있을 경우 제거
        cleaned_line = re.sub(r'<[^>]*>', '', cleaned_line).strip()
        
        # 각 키워드에 대해 매칭 시도
        for keyword, level in label_map.items():
            if cleaned_line.lower() == keyword.lower():
                prefix = "#" * level
                new_line = f"{prefix} {keyword}"
                new_lines.append(new_line)
                matched = True
                
                if show_conversions:
                    conversions.append({
                        'line_num': line_num,
                        'original': line_stripped,
                        'converted': new_line,
                        'keyword': keyword
                    })
                break
        
        if not matched:
            new_lines.append(line)
    
    if show_conversions and conversions:
        print(f"    Found {len(conversions)} header(s) to label:")
        for conv in conversions:
            print(f"      Line {conv['line_num']}: '{conv['original']}' → '{conv['converted']}'")
    
    return '\n'.join(new_lines)

### Step 3.3: 챕터별 헤더 라벨링 실행

Checkpoint 2에서 정제된 마크다운에 헤더 라벨링을 적용합니다.

In [ ]:
chapter_markdowns_labeled = {}

print("=" * 60)
print("Starting Header Labeling")
print("=" * 60)

total_conversions = 0

for chapter_name, text in checkpoint_2_cleaned_markdowns.items():
    print(f"\n[{chapter_name}]")
    print(f"  Processing header labeling...")
    
    # 헤더 라벨링 적용 (변환 내용 표시)
    labeled_text = label_headers(text, header_labels, show_conversions=True)
    
    # 변환된 텍스트 저장
    chapter_markdowns_labeled[chapter_name] = labeled_text
    
    print(f"  ✓ Completed")

print("\n" + "=" * 60)
print(f"✓ Header labeling completed for all chapters!")
print(f"  Total chapters: {len(chapter_markdowns_labeled)}")
print("=" * 60)

### Step 3.4: 체크포인트 저장 및 결과 확인

헤더가 라벨링된 마크다운을 체크포인트로 저장하고 샘플 결과를 확인합니다.

In [ ]:
# 체크포인트 3 저장
checkpoint_3_labeled_markdowns = chapter_markdowns_labeled.copy()

print("✓ Checkpoint 3 saved successfully!")
print(f"  Variable: checkpoint_3_labeled_markdowns")
print(f"  Chapters: {list(checkpoint_3_labeled_markdowns.keys())}")

# 변환 예시 찾기
sample_chapter = None
for chapter_name, text in checkpoint_3_labeled_markdowns.items():
    if "## Key Terms" in text or "## Exercises" in text:
        sample_chapter = chapter_name
        break

if sample_chapter:
    print(f"\n--- Sample: {sample_chapter} (showing Key Terms/Exercises section) ---")
    text = checkpoint_3_labeled_markdowns[sample_chapter]
    
    lines = text.split('\n')
    for i, line in enumerate(lines):
        if line.startswith("## Key Terms") or line.startswith("## Exercises"):
            sample_text = '\n'.join(lines[i:min(i+10, len(lines))])
            print(sample_text)
            print("...")
            break
else:
    print("\n--- Sample: First 500 characters of Chapter 1 ---")
    print(checkpoint_3_labeled_markdowns["Chapter 1"][:500])
    print("...")

---
## Checkpoint 4: Header-based Chunking

LangChain의 MarkdownHeaderTextSplitter를 사용하여 마크다운 헤더를 기준으로 텍스트를 청크로 분할합니다.

### 입력
- `checkpoint_3_labeled_markdowns`: 헤더가 라벨링된 챕터별 마크다운
- `headers_to_split_on`: 분할 기준이 되는 헤더 레벨 (list)

### 출력
- `chapter_chunks`: 챕터별 청크 리스트 (dict)

### 체크포인트 저장
- 변수: `checkpoint_4_chunks`

### Step 4.1: 라이브러리 설치

LangChain의 MarkdownHeaderTextSplitter를 사용하기 위한 라이브러리를 설치합니다.

In [ ]:
!pip install langchain-text-splitters -q

### Step 4.2: MarkdownHeaderTextSplitter 및 RecursiveCharacterTextSplitter 설정

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

# 분할 기준이 되는 헤더 레벨 설정
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
    ("#####", "Header 5"),
]

# MarkdownHeaderTextSplitter 생성
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
    strip_headers=False
)

# RecursiveCharacterTextSplitter 생성 (큰 청크 재분할용)
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1600,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

print("MarkdownHeaderTextSplitter Configuration:")
print(f"  Headers to split on:")
for header_symbol, metadata_key in headers_to_split_on:
    print(f"    - '{header_symbol}' → metadata key: '{metadata_key}'")

print(f"\nRecursiveCharacterTextSplitter Configuration:")
print(f"  Chunk size: 1600 characters (~400 tokens)")
print(f"  Chunk overlap: 200 characters (~50 tokens)")

### Step 4.3: 청크 분할 수행

In [ ]:
import re
from langchain_core.documents import Document

def split_large_chunk(chunk, threshold=700):
    """
    700자를 초과하는 청크를 RecursiveCharacterTextSplitter로 재분할하고
    각 서브청크에 원래 헤더 정보를 보존하며, 끊긴 문장을 병합합니다.
    """
    if len(chunk.page_content) <= threshold:
        return [chunk]

    # 메타데이터에서 헤더 정보 추출
    headers = []
    for i in range(1, 6):
        header_key = f"Header {i}"
        if header_key in chunk.metadata:
            header_symbols = "#" * i
            headers.append(f"{header_symbols} {chunk.metadata[header_key]}")
    
    header_text = "\n".join(headers) if headers else ""

    # 본문만 추출
    content = chunk.page_content
    for header in headers:
        content = content.replace(header, "").strip()

    if not content:
        return [chunk]

    # 스마트 문단 감지 전처리
    content = re.sub(r' {3,}', '\n\n', content)
    content = re.sub(r'\n([A-Z][a-z]{3,})', r'\n\n\1', content)

    # RecursiveCharacterTextSplitter로 재분할
    sub_texts = recursive_splitter.split_text(content)

    # 후처리: 소문자로 시작하는 청크 병합
    merged_texts = []
    for text in sub_texts:
        text_stripped = text.strip()
        if not text_stripped:
            continue
        
        if text_stripped[0].islower() and merged_texts:
            merged_texts[-1] = merged_texts[-1] + " " + text_stripped
        else:
            merged_texts.append(text_stripped)

    # Document 생성
    sub_chunks = []
    for sub_text in merged_texts:
        full_content = f"{header_text}\n\n{sub_text}" if header_text else sub_text
        
        sub_chunk = Document(
            page_content=full_content,
            metadata=chunk.metadata.copy()
        )
        sub_chunks.append(sub_chunk)
        
    return sub_chunks

# 메인 실행
chapter_chunks = {}

print("=" * 60)
print("Starting Header-based Chunking")
print("=" * 60)

total_chunks = 0

for chapter_name, text in checkpoint_3_labeled_markdowns.items():
    print(f"\n[{chapter_name}]")
    print(f"  Text length: {len(text):,}")
    
    # MarkdownHeaderTextSplitter로 1차 분할
    initial_chunks = markdown_splitter.split_text(text)
    print(f"  ✓ Initial split: {len(initial_chunks)} chunk(s)")
    
    # 크기 임계값 초과 시 재분할
    final_chunks = []
    
    for chunk in initial_chunks:
        if len(chunk.page_content) > 700:
            sub_chunks = split_large_chunk(chunk, threshold=700)
            final_chunks.extend(sub_chunks)
        else:
            final_chunks.append(chunk)
            
    chapter_chunks[chapter_name] = final_chunks
    total_chunks += len(final_chunks)
    
    print(f"  ✓ Final chunks: {len(final_chunks)}")

print("\n" + "=" * 60)
print(f"✓ Chunking completed!")
print(f"  Total chunks: {total_chunks}")
print("=" * 60)

### Step 4.4: 체크포인트 저장 및 결과 확인

분할된 청크를 체크포인트로 저장하고 샘플 결과를 확인합니다.

In [ ]:
# 체크포인트 4 저장
checkpoint_4_chunks = chapter_chunks.copy()

print("✓ Checkpoint 4 saved successfully!")
print(f"  Variable: checkpoint_4_chunks")
print(f"  Chapters: {list(checkpoint_4_chunks.keys())}")

# 샘플 청크 확인
sample_chapter = "Chapter 2"
if sample_chapter in checkpoint_4_chunks and checkpoint_4_chunks[sample_chapter]:
    print(f"\n--- Sample: {sample_chapter} chunks ---")
    print(f"Total chunks: {len(checkpoint_4_chunks[sample_chapter])}")
    
    for i, chunk in enumerate(checkpoint_4_chunks[sample_chapter][:5]):
        print(f"\n[Chunk {i+1}]")
        print(f"  Metadata: {chunk.metadata}")
        print(f"  Content length: {len(chunk.page_content)} chars")
        preview = chunk.page_content[:100].replace('\n', ' ')
        print(f"  Preview: {preview}...")

---
## Checkpoint 5: Header-based Filtering

특정 헤더를 가진 청크를 필터링하여 제거합니다.
(예: Attribution, Exercises 등 학습에 불필요한 섹션)

### 입력
- `checkpoint_4_chunks`: 분할된 챕터별 청크
- `filter_headers`: 제거할 헤더 키워드 리스트

### 출력
- `chapter_chunks_filtered`: 필터링된 챕터별 청크 (dict)

### 체크포인트 저장
- 변수: `checkpoint_5_filtered_chunks`

### Step 5.1: 필터링 설정

학습에 불필요한 섹션을 제거하기 위한 설정을 정의합니다.

In [ ]:
# 제거할 헤더 키워드 설정
filter_headers = header_labels  # 이전에 정의된 header_labels 사용

# 각 챕터에서 강제로 제거할 초기 청크 수
skip_first_chunks = 2

print("=" * 60)
print("Header-based Filtering Configuration")
print("=" * 60)
print(f"  Filter keywords: {len(filter_headers)}")
print(f"  Skip first N chunks: {skip_first_chunks}")

for keyword in filter_headers:
    print(f"  - {keyword}")

### Step 5.2: 필터링 함수 정의

청크의 메타데이터를 확인하여 특정 헤더를 가진 청크를 필터링합니다.

In [ ]:
def should_filter_chunk(chunk, filter_keywords):
    """
    청크의 메타데이터 내 모든 헤더를 검사하여 필터링 대상인지 확인합니다.
    """
    metadata = chunk.metadata
    filter_keywords_lower = {k.lower() for k in filter_keywords}
    
    for key, value in metadata.items():
        if key.startswith("Header"):
            if isinstance(value, str) and value.lower() in filter_keywords_lower:
                return True
    return False

print("✓ Filtering function defined")

### Step 5.3: 챕터별 청크 필터링 실행

1. 위치 기반 제거: 각 챕터의 처음 N개 청크 제거
2. 키워드 기반 제거: 특정 헤더를 포함하는 청크 제거

In [ ]:
chapter_chunks_filtered = {}

print("=" * 60)
print("Starting Header-based Filtering")
print("=" * 60)

total_original = 0
total_filtered = 0

for chapter_name, chunks in checkpoint_4_chunks.items():
    print(f"\n[{chapter_name}]")
    print(f"  Original: {len(chunks)} chunks")
    
    # Step 1: 위치 기반 제거
    remaining = chunks[skip_first_chunks:] if len(chunks) > skip_first_chunks else chunks
    
    # Step 2: 키워드 기반 필터링
    filtered = [c for c in remaining if not should_filter_chunk(c, filter_headers)]
    
    chapter_chunks_filtered[chapter_name] = filtered
    
    total_original += len(chunks)
    total_filtered += len(filtered)
    
    print(f"  Filtered: {len(filtered)} chunks")

print("\n" + "=" * 60)
print(f"✓ Filtering completed!")
print(f"  Original: {total_original} → Filtered: {total_filtered}")
print(f"  Retention: {total_filtered / total_original * 100:.1f}%")
print("=" * 60)

### Step 5.4: 체크포인트 저장 및 결과 확인

In [ ]:
# 체크포인트 5 저장
checkpoint_5_filtered_chunks = chapter_chunks_filtered.copy()

print("✓ Checkpoint 5 saved successfully!")
print(f"  Variable: checkpoint_5_filtered_chunks")
print(f"  Chapters: {list(checkpoint_5_filtered_chunks.keys())}")

# 샘플 확인
sample_chapter = "Chapter 1"
if sample_chapter in checkpoint_5_filtered_chunks:
    chunks = checkpoint_5_filtered_chunks[sample_chapter]
    print(f"\n--- Sample: {sample_chapter} ({len(chunks)} chunks) ---")
    for i, chunk in enumerate(chunks[:3]):
        print(f"\n[Chunk {i+1}]")
        print(f"  Metadata: {chunk.metadata}")
        preview = chunk.page_content[:150].replace('\n', ' ')
        print(f"  Preview: {preview}...")

---
## Checkpoint 6: Embedding with Clova

각 청크에 대해 Naver Clova 임베딩 API를 호출하여 벡터를 생성합니다.
QPM(Queries Per Minute) 제한을 고려하여 rate limiting을 적용합니다.

### 입력
- `checkpoint_5_filtered_chunks`: 필터링된 챕터별 청크
- Clova API 설정 (환경변수)

### 출력
- `chunk_embeddings`: 청크별 임베딩 벡터와 메타데이터

### 체크포인트 저장
- 변수: `checkpoint_6_embeddings`

### Step 6.1: 환경변수 및 API 설정

.env 파일에서 API 키를 로드합니다.

In [ ]:
import os
import requests
import time
from typing import List, Dict, Any
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# Clova Studio Embedding v2 API 설정
CLOVA_API_ENDPOINT = "https://clovastudio.stream.ntruss.com/v1/api-tools/embedding/v2/"
CLOVA_API_KEY = os.getenv("CLOVASTUDIO_API_KEY")

# Rate Limiting 설정
QPM_LIMIT = 60  # 분당 쿼리 제한

print("Clova Studio Embedding v2 API Configuration:")
print(f"  Endpoint: {CLOVA_API_ENDPOINT}")
print(f"  QPM Limit: {QPM_LIMIT}")
print(f"  API Key configured: {'Yes' if CLOVA_API_KEY else 'No (Please check .env)'}")

if not CLOVA_API_KEY:
    raise ValueError("CLOVASTUDIO_API_KEY not found in environment variables")

### Step 6.2: Rate Limiter 구현

QPM 제한을 준수하는 Rate Limiter를 구현합니다.

In [ ]:
from collections import deque
from datetime import datetime, timedelta

class RateLimiter:
    def __init__(self, qpm_limit: int):
        self.qpm_limit = qpm_limit
        self.request_times = deque()
    
    def wait_if_needed(self):
        now = datetime.now()
        one_minute_ago = now - timedelta(minutes=1)
        
        # 1분 이내의 요청만 유지
        while self.request_times and self.request_times[0] < one_minute_ago:
            self.request_times.popleft()
        
        # QPM 제한에 도달한 경우 대기
        if len(self.request_times) >= self.qpm_limit:
            oldest_request = self.request_times[0]
            wait_until = oldest_request + timedelta(minutes=1)
            wait_seconds = (wait_until - now).total_seconds()
            
            if wait_seconds > 0:
                print(f"    Rate limit reached. Waiting {wait_seconds:.1f}s...")
                time.sleep(wait_seconds + 0.1)
                
                now = datetime.now()
                one_minute_ago = now - timedelta(minutes=1)
                while self.request_times and self.request_times[0] < one_minute_ago:
                    self.request_times.popleft()
        
        self.request_times.append(now)

rate_limiter = RateLimiter(QPM_LIMIT)
print("✓ Rate Limiter initialized")

### Step 6.3: Clova Embedding 함수 정의

In [ ]:
def get_clova_embedding(text: str, retry_count: int = 3) -> List[float]:
    """
    Clova Studio Embedding v2 API를 호출하여 임베딩을 생성합니다.
    """
    headers = {
        "Authorization": f"Bearer {CLOVA_API_KEY}",
        "Content-Type": "application/json"
    }
    
    request_body = {"text": text}
    
    for attempt in range(retry_count):
        try:
            rate_limiter.wait_if_needed()
            
            response = requests.post(
                CLOVA_API_ENDPOINT,
                headers=headers,
                json=request_body,
                timeout=30
            )
            
            if response.status_code == 200:
                result = response.json()
                embedding = result.get("result", {}).get("embedding", [])
                if embedding:
                    return embedding
                raise Exception("No embedding in response")
            else:
                print(f"    API Error ({attempt + 1}/{retry_count}): {response.status_code}")
                if attempt < retry_count - 1:
                    time.sleep(2 ** attempt)
                    
        except Exception as e:
            print(f"    Exception ({attempt + 1}/{retry_count}): {str(e)[:100]}")
            if attempt < retry_count - 1:
                time.sleep(2 ** attempt)
    
    raise Exception(f"Failed to get embedding after {retry_count} attempts")

print("✓ Clova embedding function defined")

### Step 6.4: 청크별 임베딩 생성 실행

In [ ]:
from datetime import datetime

# 모든 청크를 평탄화
all_chunks = []
for chapter_name, chunks in checkpoint_5_filtered_chunks.items():
    for chunk_idx, chunk in enumerate(chunks):
        all_chunks.append({
            'chapter': chapter_name,
            'chunk_index': chunk_idx,
            'metadata': chunk.metadata,
            'content': chunk.page_content
        })

print("=" * 60)
print("Starting Clova Embedding Generation")
print("=" * 60)
print(f"Total chunks: {len(all_chunks)}")
print(f"Estimated time: ~{len(all_chunks) / QPM_LIMIT:.1f} minutes")
print()

chunk_embeddings = []
failed_chunks = []
start_time = datetime.now()

for idx, chunk_data in enumerate(all_chunks):
    if idx % 10 == 0:
        print(f"[{idx + 1}/{len(all_chunks)}] {chunk_data['chapter']}")
    
    try:
        embedding = get_clova_embedding(chunk_data['content'])
        chunk_embeddings.append({
            'chapter': chunk_data['chapter'],
            'chunk_index': chunk_data['chunk_index'],
            'metadata': chunk_data['metadata'],
            'content': chunk_data['content'],
            'embedding': embedding,
            'embedding_dim': len(embedding)
        })
    except Exception as e:
        print(f"  ✗ Failed: {str(e)[:50]}")
        failed_chunks.append(chunk_data)

total_time = (datetime.now() - start_time).total_seconds()

print("\n" + "=" * 60)
print(f"✓ Embedding completed!")
print(f"  Success: {len(chunk_embeddings)}, Failed: {len(failed_chunks)}")
print(f"  Total time: {total_time / 60:.1f} minutes")
print("=" * 60)

### Step 6.5: 체크포인트 저장

In [ ]:
import json

# 체크포인트 6 저장
checkpoint_6_embeddings = chunk_embeddings.copy()

# JSON 파일로 저장
embeddings_for_json = []
for item in checkpoint_6_embeddings:
    embeddings_for_json.append({
        'chapter': item['chapter'],
        'chunk_index': item['chunk_index'],
        'metadata': item['metadata'],
        'content': item['content'],
        'embedding': item['embedding'],
        'embedding_dim': item['embedding_dim']
    })

json_file_path = 'checkpoint_6_embeddings.json'
with open(json_file_path, 'w', encoding='utf-8') as f:
    json.dump(embeddings_for_json, f, ensure_ascii=False, indent=2)

print(f"✓ Checkpoint 6 saved to {json_file_path}")
print(f"  Total embeddings: {len(checkpoint_6_embeddings)}")

if checkpoint_6_embeddings:
    sample = checkpoint_6_embeddings[0]
    print(f"\n--- Sample ---")
    print(f"  Chapter: {sample['chapter']}")
    print(f"  Embedding dim: {sample['embedding_dim']}")
    print(f"  Embedding preview: {sample['embedding'][:5]}...")

---
## Checkpoint 7: PostgreSQL pgvector Loading

생성된 임베딩 벡터를 PostgreSQL의 pgvector extension을 사용하여 데이터베이스에 적재합니다.

### 입력
- `checkpoint_6_embeddings`: 청크별 임베딩 벡터와 메타데이터
- PostgreSQL 연결 정보 (환경변수)

### 출력
- 데이터베이스에 적재 완료

### Step 7.1: 라이브러리 설치 및 임포트

In [ ]:
!pip install psycopg2-binary -q

In [ ]:
import psycopg2
from psycopg2.extras import execute_batch
import json

print("✓ Libraries imported")

### Step 7.2: PostgreSQL 연결 설정

.env 파일에서 DB 연결 정보를 로드합니다.

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# PostgreSQL 연결 정보 (환경변수에서 로드)
PG_HOST = os.getenv("DB_HOST", "localhost")
PG_PORT = int(os.getenv("DB_PORT", "5432"))
PG_DATABASE = os.getenv("DB_NAME", "mydb")
PG_USER = os.getenv("DB_USER", "myuser")
PG_PASSWORD = os.getenv("DB_PASSWORD")

# 테이블 설정
TABLE_NAME = "document_embeddings"
CATEGORY = "Network"

print("PostgreSQL Configuration:")
print(f"  Host: {PG_HOST}")
print(f"  Port: {PG_PORT}")
print(f"  Database: {PG_DATABASE}")
print(f"  User: {PG_USER}")
print(f"  Password: {'*' * len(PG_PASSWORD) if PG_PASSWORD else 'Not set'}")
print(f"  Table: {TABLE_NAME}")

if not PG_PASSWORD:
    print("\n⚠ Warning: DB_PASSWORD not found in environment variables")

### Step 7.3: 테이블 스키마 생성

pgvector extension과 테이블을 생성합니다.

In [ ]:
try:
    conn = psycopg2.connect(
        host=PG_HOST,
        port=PG_PORT,
        database=PG_DATABASE,
        user=PG_USER,
        password=PG_PASSWORD
    )
    conn.autocommit = True
    cursor = conn.cursor()
    
    print("✓ Connected to PostgreSQL")
    
    # pgvector extension 생성
    cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    print("✓ pgvector extension enabled")
    
    # 테이블 존재 여부 확인
    cursor.execute(f"""
        SELECT EXISTS (
            SELECT FROM information_schema.tables 
            WHERE table_name = '{TABLE_NAME}'
        );
    """)
    table_exists = cursor.fetchone()[0]
    
    if table_exists:
        print(f"⚠ Table '{TABLE_NAME}' exists. Truncating...")
        cursor.execute(f"TRUNCATE TABLE {TABLE_NAME};")
    else:
        print(f"Creating table '{TABLE_NAME}'...")
        cursor.execute(f"""
        CREATE TABLE {TABLE_NAME} (
            id SERIAL PRIMARY KEY,
            content TEXT NOT NULL,
            category VARCHAR(255) NOT NULL,
            embedding VECTOR(1024) NOT NULL,
            tsvector TSVECTOR,
            metadata JSONB,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """)
        
        # 인덱스 생성
        cursor.execute(f"""
            CREATE INDEX {TABLE_NAME}_embedding_idx 
            ON {TABLE_NAME} USING hnsw (embedding vector_cosine_ops);
        """)
        cursor.execute(f"""
            CREATE INDEX {TABLE_NAME}_tsvector_idx 
            ON {TABLE_NAME} USING gin (tsvector);
        """)
        cursor.execute(f"""
            CREATE INDEX {TABLE_NAME}_category_idx 
            ON {TABLE_NAME} (category);
        """)
        print("✓ Table and indexes created")
    
    print("✓ Database setup completed")
    
except Exception as e:
    print(f"✗ Database setup failed: {str(e)}")
    raise

### Step 7.4: 배치 INSERT 실행

임베딩 데이터를 100개씩 배치로 PostgreSQL에 적재합니다.

In [ ]:
from datetime import datetime

BATCH_SIZE = 100

print("=" * 60)
print("Starting Batch INSERT")
print("=" * 60)
print(f"Total embeddings: {len(checkpoint_6_embeddings)}")
print(f"Batch size: {BATCH_SIZE}")
print()

insert_sql = f"""
    INSERT INTO {TABLE_NAME} (content, category, embedding, tsvector, metadata)
    VALUES (%s, %s, %s, to_tsvector('english', %s), %s);
"""

inserted_count = 0
start_time = datetime.now()

try:
    for batch_start in range(0, len(checkpoint_6_embeddings), BATCH_SIZE):
        batch_end = min(batch_start + BATCH_SIZE, len(checkpoint_6_embeddings))
        batch = checkpoint_6_embeddings[batch_start:batch_end]
        
        batch_num = (batch_start // BATCH_SIZE) + 1
        print(f"[Batch {batch_num}] Inserting rows {batch_start + 1} to {batch_end}...")
        
        batch_data = []
        for item in batch:
            batch_data.append((
                item['content'],
                CATEGORY,
                item['embedding'],
                item['content'],
                json.dumps(item['metadata'])
            ))
        
        execute_batch(cursor, insert_sql, batch_data, page_size=BATCH_SIZE)
        inserted_count += len(batch_data)
        print(f"  ✓ Inserted {len(batch_data)} rows")
    
    conn.commit()
    
    total_time = (datetime.now() - start_time).total_seconds()
    print("\n" + "=" * 60)
    print(f"✓ INSERT completed!")
    print(f"  Inserted: {inserted_count} rows")
    print(f"  Time: {total_time:.1f}s")
    print("=" * 60)
    
except Exception as e:
    print(f"✗ INSERT failed: {str(e)}")
    conn.rollback()
    raise

### Step 7.5: 데이터 검증 및 결과 확인

In [ ]:
print("=" * 60)
print("Data Validation")
print("=" * 60)

try:
    # 총 레코드 수
    cursor.execute(f"SELECT COUNT(*) FROM {TABLE_NAME};")
    total_count = cursor.fetchone()[0]
    print(f"\nTotal records: {total_count}")
    
    # 카테고리별 통계
    cursor.execute(f"""
        SELECT category, COUNT(*) 
        FROM {TABLE_NAME} GROUP BY category;
    """)
    for category, count in cursor.fetchall():
        print(f"  {category}: {count}")
    
    # 샘플 데이터
    cursor.execute(f"""
        SELECT id, LEFT(content, 80) as preview, category
        FROM {TABLE_NAME} ORDER BY id LIMIT 3;
    """)
    print("\n--- Sample Records ---")
    for record_id, preview, category in cursor.fetchall():
        print(f"[{record_id}] {category}: {preview}...")
    
    # 테이블 크기
    cursor.execute(f"SELECT pg_size_pretty(pg_total_relation_size('{TABLE_NAME}'));")
    print(f"\nTable size: {cursor.fetchone()[0]}")
    
    print("\n" + "=" * 60)
    print("✓ Validation completed!")
    print("=" * 60)
    
except Exception as e:
    print(f"✗ Validation failed: {str(e)}")
finally:
    cursor.close()
    conn.close()
    print("\n✓ Database connection closed")